In [1]:
!pip install transformers -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 31.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.37.2
    Uninstalling transformers-4.37.2:
      Successfully uninstalled transformers-4.37.2


In [23]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 5.2 MB/s eta 0:00:00


In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Reading Data

In [3]:
df=pd.read_json('/content/drive/MyDrive/Data/cancelled.json')
dfv0=pd.read_json('/content/drive/MyDrive/Data/sailorapp.json')
dfv1=pd.read_json('/content/drive/MyDrive/Data/noncancelled.json')

In [4]:
df['type']='invalid'
dfv1['type']='valid'
dfv0['type']='valid'
dfv0['Component']=' '
df_combined=pd.concat([df,dfv0,dfv1])

**Preprocessing**

In [5]:
# Assuming df is your DataFrame
# Group by 'issuekey' and count occurrences
issuekey_counts = df_combined.groupby('IssueKey').size()

# Filter the DataFrame where count values are greater than 1
filtered_df = df_combined[df_combined['IssueKey'].isin(issuekey_counts[issuekey_counts > 1].index)]
canceled=filtered_df[filtered_df['Status']=='Canceled']
canceled_list=canceled['IssueKey'].to_list()
df_total=df_combined[~df_combined['IssueKey'].isin(canceled_list)]
df_total['text'] = df_total['Description'] + ' ' + df_total['Summary']

<ipython-input-5-0b4e026f2e39>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_total['text'] = df_total['Description'] + ' ' + df_total['Summary']


In [6]:
df_total['type']=df_total['type'].apply(lambda x: 1 if x=='valid' else 0)
df_total_new=df_total.drop_duplicates(subset='text')

<ipython-input-6-16e74cb2f603>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_total['type']=df_total['type'].apply(lambda x: 1 if x=='valid' else 0)


In [7]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Ensure that the necessary NLTK data is downloaded
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

def clean_text(text):
    """Clean raw text string, remove stopwords, and apply lemmatization."""
    # Initialize WordNet Lemmatizer
    lemmatizer = WordNetLemmatizer()

    # Define English stopwords
    stop_words = set(stopwords.words('english'))

    # Remove patterns like [~accountid:...] and numbers
    text = re.sub(r'\[~accountid:\w+\]|\d+', '', text)

    # Remove URLs
    text = re.sub(r'http\S+', '', text)

    # Remove unnecessary characters
    text = re.sub(r"[\r\n]+", " ", text)  # Replace newlines with spaces
    text = re.sub(r"[^\w\s]+", " ", text)  # Remove non-alphanumeric characters except spaces
    text = re.sub(r"\s+", " ", text)  # Remove multiple spaces

    # Strip leading and trailing whitespace
    text = text.strip()

    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stopwords and lemmatize the words
    filtered_tokens = [lemmatizer.lemmatize(word.lower()) for word in tokens if word.lower() not in stop_words]

    # Join the cleaned tokens back into a string
    clean_text = ' '.join(filtered_tokens)

    return clean_text


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
df_total_new['text'] = df_total_new['text'].apply(lambda x: clean_text(x))

<ipython-input-9-c0d3187e78ba>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_total_new['text'] = df_total_new['text'].apply(lambda x: clean_text(x))


In [10]:
df=df_total_new.reset_index()

**Pretrained** **Bert**

In [11]:
from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [12]:
model = model.to('cuda')

In [13]:
sample_data = [" Incorrect SA logo on app launch when some process are launch on phone header"]
tokenizer(sample_data, padding=True, truncation=True, max_length=512)

{'input_ids': [[101, 16542, 7842, 8154, 2006, 10439, 4888, 2043, 2070, 2832, 2024, 4888, 2006, 3042, 20346, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

**Data Split**

In [12]:
X = list(df["text"])
y = list(df["type"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

In [15]:

X_train_tokenized.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [16]:
print(X_train_tokenized['attention_mask'][0])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [13]:
len(X_train),len(X_val)

(6447, 1612)

In [14]:

# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [15]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [20]:
train_dataset[2]

{'input_ids': tensor([  101,  5310,  2171,  9587, 19496,  2912,  7959,  2497, 10930,  9737,
         12502,  4012,  4748, 10020,  3653,  8663, 20562, 11803, 11299,  6080,
          4576,  8354, 16215, 13114,  8833,  2378, 11803, 10439,  2911,  2897,
         20410,  3746, 20327, 14779,  2930,  3491,  3277,  3746, 20327, 14779,
          2930,  3517,  3746, 20327, 14779,  2930,  4487, 13102, 20766,  2098,
         11803, 10439,  1054, 20327,  2678,  1035,  2053, 14779,  3746, 20327,
          2678,  2930,   102,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [16]:
def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [18]:

# Define Trainer
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=5,
    per_device_train_batch_size=8

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [19]:
trainer.train()

Step,Training Loss
500,0.605800
1000,0.620700
1500,0.581400
2000,0.555800
2500,0.515500
3000,0.485800
3500,0.467800
4000,0.459900


TrainOutput(global_step=4030, training_loss=0.5356867397393541, metrics={'train_runtime': 1003.5141, 'train_samples_per_second': 32.122, 'train_steps_per_second': 4.016, 'total_flos': 8481384869529600.0, 'train_loss': 0.5356867397393541, 'epoch': 5.0})

Perfomance Scores

In [20]:
trainer.evaluate()

<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.5167102217674255,
 'eval_accuracy': 0.7921836228287841,
 'eval_precision': 0.7988121752041574,
 'eval_recall': 0.9438596491228071,
 'eval_f1': 0.8652995577000401,
 'eval_runtime': 15.9337,
 'eval_samples_per_second': 101.169,
 'eval_steps_per_second': 12.678,
 'epoch': 5.0}

In [21]:
np.set_printoptions(suppress=True)

**Inference**

In [22]:
text = "When attempting to open prod build of application via vpn user is show red loaing screen for a short time then is met with a configuration error "
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
outputs = model(**inputs)
print(outputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)
predictions = predictions.cpu().detach().numpy()
predictions

SequenceClassifierOutput(loss=None, logits=tensor([[-1.2100,  0.3823]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[0.1691, 0.8309]], device='cuda:0', grad_fn=<SoftmaxBackward0>)


array([[0.16905189, 0.8309481 ]], dtype=float32)

In [25]:
text = "SAPP | QA | A message to enable camera permission is not displayed while trying to take picture for Security ID task if the permission is in disabled state "
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
outputs = model(**inputs)
print(outputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)
predictions = predictions.cpu().detach().numpy()
predictions

SequenceClassifierOutput(loss=None, logits=tensor([[ 2.1388, -0.8759]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[0.9532, 0.0468]], device='cuda:0', grad_fn=<SoftmaxBackward0>)


array([[0.9532307 , 0.04676936]], dtype=float32)

**Deploy**

In [26]:
from huggingface_hub import notebook_login

notebook_login()

In [27]:
model.push_to_hub("MaheshPrabhu/FineTune_Bert")

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/MaheshPrabhu/FineTune_Bert/commit/6af66a558db4fb3a09108b43a49a913ac3d05281', commit_message='Upload BertForSequenceClassification', commit_description='', oid='6af66a558db4fb3a09108b43a49a913ac3d05281', pr_url=None, pr_revision=None, pr_num=None)

In [2]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

In [3]:
model = BertForSequenceClassification.from_pretrained("MaheshPrabhu/FineTune_Bert")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]